# Mining and Classifiers

In [158]:
import importlib
import utils 
importlib.reload(utils)
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils
import plot_utils
importlib.reload(plot_utils)
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 
import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, MyRandomClassifier, MyZeroRClassifier, MyDecisionTreeClassifier, MyRandomForestClassifier
from tabulate import tabulate

music_table = MyPyTable()
music_fname = os.path.join("input_data", "music-data.csv")
music_table.load_from_file(music_fname)

music_data = music_table.data.copy()
del_list = [0, 0, 2]
for j in music_data:
    for i in del_list:
        del j[i]
# do holdout method on dataset - put testing set aside
# remainder set has 2/3 of original
# train n ensemble learners of subset of remainder set
header = ["level", "lang", "tweets", "phd", "interviewed_well"]
table = [
    ["Senior", "Java", "no", "no", "False"],
    ["Senior", "Java", "no", "yes", "False"],
    ["Mid", "Python", "no", "no", "True"],
    ["Junior", "Python", "no", "no", "True"],
    ["Junior", "R", "yes", "no", "True"],
    ["Junior", "R", "yes", "yes", "False"],
    ["Mid", "R", "yes", "yes", "True"],
    ["Senior", "Python", "no", "no", "False"],
    ["Senior", "R", "yes", "no", "True"],
    ["Junior", "Python", "yes", "no", "True"],
    ["Senior", "Python", "yes", "yes", "True"],
    ["Mid", "Python", "no", "yes", "True"],
    ["Mid", "Java", "yes", "no", "True"],
    ["Junior", "Python", "no", "yes", "False"]
]

for i, value in enumerate(music_data):
    if value[0] <= 20:
        value[0] = "Low"
    elif value[0] > 20 and value[0] <= 40:
        value[0] = "Below Average"
    elif value[0] > 40 and value[0] <= 60:
        value[0] = "Average"
    elif value[0] > 60 and value[0] <= 80:
        value[0] = "Above Average"
    elif value[0] > 80:
        value[0] = "High"


duration_col = myutils.get_column_by_index(music_data, 1)
cutoffs = myutils.compute_equal_width_cutoffs(duration_col, 3)
for i, value in enumerate(music_data):
    if value[1] <= 227131.33:
        value[1] = "short"
    elif value[1] > 227131.33 and value[1] <= 413062.67:
        value[1] = "medium"
    elif value[1] > 413062.67 and value[1] <= 598994:
        value[1] = "long"

markets_col = myutils.get_column_by_index(music_data, 4)
cutoffs = myutils.compute_equal_width_cutoffs(markets_col, 3)
for i, value in enumerate(music_data):
    if value[4] <= 60.0:
        value[4] = "small"
    elif value[4] > 60.0 and value[4] <= 119.0:
        value[4] = "medium"
    elif value[4] > 119.0:
        value[4] = "large"

myutils.combine_genre(music_data)

danceability_col = myutils.get_column_by_index(music_data, 5)
cutoffs = myutils.compute_equal_width_cutoffs(danceability_col, 3)
for i, value in enumerate(music_data):
    if value[5] <= 0.32:
        value[5] = "1"
    elif value[5] > 0.32 and value[5] <= 0.65:
        value[5] = "2"
    elif value[5] > 0.65:
        value[5] = "3"

acousticness_col = myutils.get_column_by_index(music_data, 6)
cutoffs = myutils.compute_equal_width_cutoffs(acousticness_col, 3)
for i, value in enumerate(music_data):
    if value[6] <= 0.33:
        value[6] = "1"
    elif value[6] > 0.33 and value[6] <= 0.66:
        value[6] = "2"
    elif value[6] > 0.66:
        value[6] = "3"

tempo_col = myutils.get_column_by_index(music_data, 7)
cutoffs = myutils.compute_equal_width_cutoffs(tempo_col, 3)
for i, value in enumerate(music_data):
    if value[7] <= 71.34:
        value[7] = "1"
    elif value[7] > 71.34 and value[7] <= 142.68:
        value[7] = "2"
    elif value[7] > 142.68:
        value[7] = "3"
# music_table.data = table
# music_table.column_names = header

m_labels, m_table = myutils.get_train_labels(music_data, 0)
label_list = []
for val in m_labels:
    if val not in label_list:
        label_list.append(val)
X_train, X_test, y_train, y_test = myevaluation.train_test_split(m_table, m_labels)
label_list.sort()

naive = MyNaiveBayesClassifier()
naive.fit(X_train, y_train)
y_pred_list = []
for val in X_test:
    y_pred_list.append(naive.predict(val))
matrix = myevaluation.confusion_matrix(list(y_test), y_pred_list, label_list)
for i, val in enumerate(matrix):
    val.insert(0, label_list[i])
print("===========================================")
print("Music Data - Naive Bayes Results")
print("===========================================")
print(tabulate(matrix, headers = label_list))
accuracy = myutils.compute_accuracy(matrix)
print("Accuracy = ", accuracy)
error_rate = myutils.compute_error_rate(matrix)
print("Error Rate = ", error_rate)

dec_tree = MyDecisionTreeClassifier()
dec_tree.fit(X_train, y_train)
y_pred_list = dec_tree.predict(X_test)

matrix = myevaluation.confusion_matrix(list(y_test), y_pred_list, label_list)
for i, val in enumerate(matrix):
    val.insert(0, label_list[i])

print("===========================================")
print("Music Data - Decision Tree Results")
print("===========================================")
print(tabulate(matrix, headers = label_list))
accuracy = myutils.compute_accuracy(matrix)
print("Accuracy = ", accuracy)
error_rate = myutils.compute_error_rate(matrix)
print("Error Rate = ", error_rate)



random_forest = MyRandomForestClassifier()
random_forest.fit(X_train, y_train, music_table.column_names, label_list)
pred_list = random_forest.predict(X_test[:1])
# matrix = myevaluation.confusion_matrix(list(y_test), pred_list, label_list)
# for i, val in enumerate(matrix):
#     val.insert(0, label_list[i])





Music Data - Naive Bayes Results
                 Above Average    Average    Below Average    High    Low
-------------  ---------------  ---------  ---------------  ------  -----
Above Average                0          0                0     402      6
Average                      1          0                0      20      0
Below Average                0          0                0       5      0
High                         0          0                0      39      2
Low                          0          0                0      69      5
Accuracy =  0.08014571948998178
Error Rate =  0.04735883424408015
Music Data - Decision Tree Results
                 Above Average    Average    Below Average    High    Low
-------------  ---------------  ---------  ---------------  ------  -----
Above Average              400          0                0       0      8
Average                     19          0                0       0      2
Below Average                5          0           